In [1]:
import numpy as np
import torch

In [2]:
torch.manual_seed(100)
d = 10   # original dimension
k = 2    # reduced dimension
N = 100  # number of data points
X = torch.rand((d, N))
X = (X.T - torch.mean(X, 1)).T
U, S, V = torch.svd(X)
U = U[:, :k]  # only the top k singular vectors
U.requires_grad_(True)

def mm(a, b):
    return torch.matmul(a, b)

residual = mm((torch.eye(d) - mm(U, U.T)), X)
if U.grad is not None:
    U.grad.zero_()
f = torch.norm(residual, p="fro") ** 2
if U.grad is not None:
    U.grad.zero_()
res = torch.autograd.grad(f, U)
U_grad = res[0]

In [3]:
print(U.shape)

torch.Size([10, 2])


In [3]:
if U.grad is not None:
    U.grad.zero_()
f_analytical = torch.norm(X, p="fro") ** 2 - torch.norm(U.T @ X, p="fro") ** 2
if U.grad is not None:
    U.grad.zero_()
with torch.no_grad():
    U_grad_analytical = - 2 * X @ X.T @ U.clone()
U_grad_analytical_backward, = torch.autograd.grad(f_analytical, U)
with torch.no_grad():
    print(torch.dist(f, f_analytical))
print(torch.dist(U_grad_analytical_backward, U_grad_analytical))
print(torch.dist(U_grad_analytical_backward, U_grad))
print(torch.dist(U_grad_analytical, U_grad))

tensor(3.0518e-05)
tensor(4.7208e-06)
tensor(32.7275)
tensor(32.7275)


In [4]:
import torchviz
from torchviz import make_dot

In [5]:
make_dot(f)

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH